# Causal Inference Analysis: Effect of High Discount Percentage on Product Star Rating

**Objective**: To estimate the causal effect of a product having a "High Discount Percentage" on its average customer star `rating` on MercadoLibre.

**Methodology**: We will use Propensity Score Matching (PSM) to create comparable groups of products (those with a high discount vs. those with a low/no discount) within a specific product category. This will help control for confounding variables such as current price, free delivery status, seller verification, total review count, and the presence of additional discount coupons.

**Outcome Variable (Y)**: `rating` (Product star rating)
**Treatment Variable (X)**: `is_high_discount` (Binary: 1 if discount is high, 0 if low/none)
**Covariates (Z)**: `current_price`, `has_free_delivery`, `is_verified_seller`, `total_review_count`, `has_additional_discount_coupon`

**Data Source**: Scraped data from MercadoLibre products.

In [1]:
import pandas as pd
import numpy as np
# np.float = float # Keep this if needed for older packages, otherwise, ensure your libraries are up to date.
                # Modern numpy versions use np.float_ or just float.
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
import statsmodels.api as sm
from scipy.stats import ttest_ind, chi2_contingency # Added chi2_contingency for binary covariates
from statsmodels.stats.proportion import proportions_ztest # Alternative for binary

# For consistent styling of outputs (optional)
pd.options.display.float_format = '{:.3f}'.format

## Load, Inspect, and Prepare Data

1.  **Load the dataset**: Update with your actual filename.
2.  **Filter for a specific Product Category**: This is crucial for meaningful analysis.
3.  **Create Discount Percentage Variable**: If not already scraped directly.
4.  **Create Binary Treatment Variable (`is_high_discount`)**.
5.  **Rename columns for clarity** if needed (e.g., `new_price` to `current_price`, `review_count` to `total_review_count`).
6.  **Handle Missing Values**: Address NaNs in treatment, outcome, and covariates.

In [2]:

# Define the input CSV path
DATA_CSV_PATH = 'data/trialrun/mercadolibre_solar_cream.csv'

print(f"Loading dataset from: {DATA_CSV_PATH}")
try:
    df = pd.read_csv(DATA_CSV_PATH)
    if df.empty:
        raise ValueError("Loaded DataFrame is empty. Please check the CSV file.")
    print(f"Successfully loaded dataset. Shape: {df.shape}")
except FileNotFoundError:
    raise FileNotFoundError(f"Error: Dataset file '{DATA_CSV_PATH}' not found. Please ensure the file is in the correct location and you have uploaded it.")

# Ensure price columns are numeric
df['old_price'] = pd.to_numeric(df['old_price'], errors='coerce')
df['new_price'] = pd.to_numeric(df['new_price'], errors='coerce')

# 1. Calculate 'discount_percentage'
df['discount_percentage'] = 0.0
# Calculate discount only where old_price is valid, positive, and greater than new_price
valid_discount_mask = (df['old_price'].notna()) & \
                    (df['new_price'].notna()) & \
                    (df['old_price'] > 0) & \
                    (df['old_price'] > df['new_price'])
df.loc[valid_discount_mask, 'discount_percentage'] = \
    ((df['old_price'] - df['new_price']) / df['old_price']) * 100
df['discount_percentage'].fillna(0, inplace=True) # Ensure no NaNs if prices were not coercible
print("\nCalculated 'discount_percentage'.")

# 2. Create 'has_free_delivery' (binary)
# Based on the 'free_shipping' column
df['has_free_delivery'] = np.where(
    df['free_shipping'].astype(str).str.contains("Envío gratis", case=False, na=False), 1, 0
)
print("Created 'has_free_delivery' column.")

# 3. Create 'is_verified_seller' (binary)
# Based on the 'official_seller' column: 0 if "Not Found", 1 otherwise
df['is_verified_seller'] = np.where(
    df['official_seller'].astype(str).str.strip().fillna("Not Found").str.upper() == "NOT FOUND", 0, 1
)
print("Created 'is_verified_seller' column.")

# 4. Create 'has_additional_discount_coupon' (binary)
# Based on the 'discount' column in the CSV (not the percentage calculated above)
# Assumes any non-zero value in 'discount' column indicates an additional distinct discount/coupon
if 'discount' in df.columns:
    df['has_additional_discount_coupon'] = np.where(pd.to_numeric(df['discount'], errors='coerce').fillna(0) > 0, 1, 0)
    print("Created 'has_additional_discount_coupon' column.")
else:
    print("Warning: Original 'discount' column not found for 'has_additional_discount_coupon'. Column set to 0.")
    df['has_additional_discount_coupon'] = 0

# 5. Rename columns for PSM analysis
df.rename(columns={
    'new_price': 'current_price',
    'review_count': 'total_review_count'
}, inplace=True)
print("Renamed columns: 'new_price' to 'current_price', 'review_count' to 'total_review_count'.")

# 6. Create Binary Treatment Variable (`is_high_discount`)
# Define "high discount" based on the median of products *that actually have a discount*
positive_discounts = df[df['discount_percentage'] > 0]['discount_percentage']
if not positive_discounts.empty:
    DISCOUNT_THRESHOLD = positive_discounts.median()
else:
    DISCOUNT_THRESHOLD = 0 # Fallback if no products have any discount
    print(f"Warning: No products with positive 'discount_percentage' values found. Threshold for 'is_high_discount' set to {DISCOUNT_THRESHOLD:.2f}%.")

df['is_high_discount'] = (df['discount_percentage'] > DISCOUNT_THRESHOLD).astype(int)
print(f"\nDiscount threshold for 'is_high_discount': {DISCOUNT_THRESHOLD:.2f}%")
print(f"Distribution of 'is_high_discount':\n{df['is_high_discount'].value_counts(dropna=False)}")

# 7. Define outcome, treatment, and covariate lists
outcome_variable = 'rating'
treatment_variable = 'is_high_discount'
covariates_list = [
    'current_price',
    'has_free_delivery',
    'is_verified_seller',
    'total_review_count',
    'has_additional_discount_coupon'
]

# Ensure 'rating' column is numeric and other key columns are correctly typed
if outcome_variable not in df.columns:
    raise ValueError(f"Outcome variable '{outcome_variable}' not found in DataFrame.")
df[outcome_variable] = pd.to_numeric(df[outcome_variable], errors='coerce')

for col in ['current_price', 'total_review_count']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else:
        # This should not happen if renaming was successful and columns existed
        raise ValueError(f"Essential covariate '{col}' is missing after renaming.")

# 8. Prepare final DataFrame for analysis (selecting analysis columns and handling NaNs)
all_analysis_columns = [outcome_variable, treatment_variable] + covariates_list
df_analysis = df[all_analysis_columns].copy()

print(f"\nShape of analysis DataFrame before dropping NaNs: {df_analysis.shape}")
df_analysis.dropna(inplace=True) # Drop rows if any of the essential columns have NaN
print(f"Shape of analysis DataFrame after dropping NaNs: {df_analysis.shape}")

# 9. Final checks for PSM readiness
if df_analysis.empty:
    print("Warning: DataFrame for analysis is empty after processing and NaN removal. PSM cannot proceed.")
else:
    if df_analysis.shape[0] < 30:
        print(f"Warning: Sample size for analysis is very small ({df_analysis.shape[0]}). PSM results may be unreliable.")
    if df_analysis[treatment_variable].nunique() < 2:
        raise ValueError(
            f"Treatment variable '{treatment_variable}' does not have at least two distinct values (0 and 1) "
            f"in the cleaned analysis data. Current value counts:\n{df_analysis[treatment_variable].value_counts(dropna=False)}\n"
            f"PSM cannot proceed. This might be due to all products having low/no discount based on the threshold."
        )

print(f"\nFinal prepared DataFrame for PSM (df_analysis) head:")
if not df_analysis.empty:
    print(df_analysis.head())
    print("\nInfo for df_analysis:")
    df_analysis.info()
else:
    print("df_analysis is empty.")

# You can now use df_analysis for the subsequent PSM steps.
# If you need to save this specific prepped DataFrame:
# df_analysis.to_csv('prepped_data_for_psm.csv', index=False)

Loading dataset from: data/trialrun/mercadolibre_solar_cream.csv
Successfully loaded dataset. Shape: (633, 10)

Calculated 'discount_percentage'.
Created 'has_free_delivery' column.
Created 'is_verified_seller' column.
Created 'has_additional_discount_coupon' column.
Renamed columns: 'new_price' to 'current_price', 'review_count' to 'total_review_count'.

Discount threshold for 'is_high_discount': 15.04%
Distribution of 'is_high_discount':
is_high_discount
0    347
1    286
Name: count, dtype: int64

Shape of analysis DataFrame before dropping NaNs: (633, 7)
Shape of analysis DataFrame after dropping NaNs: (633, 7)

Final prepared DataFrame for PSM (df_analysis) head:
   rating  is_high_discount  current_price  has_free_delivery  \
0   4.400                 1         52.000                  0   
1   5.000                 1        489.000                  1   
2   5.000                 0        522.000                  1   
3   5.000                 1        182.000                  0  

## Define Key Variables for the Analysis

* **Outcome (Y)**: `rating` (Product's average star rating, continuous)
* **Treatment (X)**: `is_high_discount` (Binary: 1 if the product has a discount percentage above the category threshold, 0 otherwise)
* **Covariates (Z)**:
    * `current_price` (Current selling price of the product)
    * `has_free_delivery` (Binary: 1 if free delivery is offered, 0 otherwise)
    * `is_verified_seller` (Binary: 1 if the seller is verified, 0 otherwise)
    * `total_review_count` (Total number of reviews for the product)
    * `has_additional_discount_coupon` (Binary: 1 if an additional coupon is offered, 0 otherwise)

    ## Step 1: Check Covariate Balance Before Matching

We will compare the means of continuous covariates and proportions of binary covariates between the 'High Discount' group and the 'Low/No Discount' group *before* applying propensity score matching. This helps us understand the initial differences between the groups and the necessity of matching.

In [3]:
# Separate treated (is_high_discount == 1) and control (is_high_discount == 0) groups
treated_group_pre_match = df[df[treatment_variable] == 1]
control_group_pre_match = df[df[treatment_variable] == 0]

if treated_group_pre_match.empty or control_group_pre_match.empty:
    raise ValueError(f"Not enough data in treatment or control group for category '{CATEGORY_TO_ANALYZE}' to proceed with balance checks.")

print(f"Pre-match - Treated group size: {len(treated_group_pre_match)}")
print(f"Pre-match - Control group size: {len(control_group_pre_match)}")


# --- Balance for Continuous Covariates ---
continuous_covariates = ['current_price', 'total_review_count']
print("\n--- Continuous Covariate Balance Before Matching ---")
balance_results_continuous_pre = []
for var in continuous_covariates:
    if var in df.columns:
        # Ensure there's data to compare
        if not treated_group_pre_match[var].dropna().empty and not control_group_pre_match[var].dropna().empty:
            mean_treated = treated_group_pre_match[var].mean()
            mean_control = control_group_pre_match[var].mean()
            std_treated = treated_group_pre_match[var].std()
            std_control = control_group_pre_match[var].std()
            
            # Standardized Mean Difference (SMD)
            # Pooled standard deviation for SMD calculation
            s_pooled = np.sqrt(((len(treated_group_pre_match)-1)*(std_treated**2) + \
                                (len(control_group_pre_match)-1)*(std_control**2)) / \
                            (len(treated_group_pre_match) + len(control_group_pre_match) - 2))
            smd = (mean_treated - mean_control) / s_pooled if s_pooled > 0 else 0
            
            t_stat, p_val = ttest_ind(treated_group_pre_match[var].dropna(), control_group_pre_match[var].dropna(), equal_var=False) # Welch's t-test
            balance_results_continuous_pre.append({
                'Covariate': var,
                'Mean Treated': f"{mean_treated:.2f}",
                'Mean Control': f"{mean_control:.2f}",
                'Std Diff (SMD)': f"{smd:.3f}", # SMD is often preferred over p-values for balance
                'T-Statistic': f"{t_stat:.2f}",
                'P-Value': f"{p_val:.3f}"
            })
        else:
            print(f"Skipping t-test for {var} due to insufficient data in one or both groups.")
    else:
        print(f"Warning: Continuous covariate '{var}' not found in DataFrame.")

if balance_results_continuous_pre:
    balance_df_continuous_pre = pd.DataFrame(balance_results_continuous_pre)
    print(balance_df_continuous_pre)
else:
    print("No continuous covariates to check or insufficient data.")


# --- Balance for Binary Covariates ---
binary_covariates = ['has_free_delivery', 'is_verified_seller', 'has_additional_discount_coupon']
print("\n--- Binary Covariate Balance Before Matching ---")
balance_results_binary_pre = []
for var in binary_covariates:
    if var in df.columns:
        # Ensure there's data to compare
        if not treated_group_pre_match[var].dropna().empty and not control_group_pre_match[var].dropna().empty:
            prop_treated = treated_group_pre_match[var].mean() # Proportion of 1s
            prop_control = control_group_pre_match[var].mean()
            
            count_treated = treated_group_pre_match[var].sum()
            n_treated = treated_group_pre_match[var].count()
            count_control = control_group_pre_match[var].sum()
            n_control = control_group_pre_match[var].count()

            if n_treated > 0 and n_control > 0: # Ensure non-zero observations for z-test
                # Standardized Mean Difference (SMD) for binary variables
                smd_binary = (prop_treated - prop_control) / np.sqrt((prop_treated * (1 - prop_treated) + prop_control * (1 - prop_control)) / 2)

                z_stat, p_val = proportions_ztest([count_treated, count_control], [n_treated, n_control])
                balance_results_binary_pre.append({
                    'Covariate': var,
                    'Prop Treated': f"{prop_treated:.2f}",
                    'Prop Control': f"{prop_control:.2f}",
                    'Std Diff (SMD)': f"{smd_binary:.3f}",
                    'Z-Statistic': f"{z_stat:.2f}",
                    'P-Value': f"{p_val:.3f}"
                })
            else:
                print(f"Skipping z-test for {var} due to zero observations in one group.")
        else:
            print(f"Skipping balance check for {var} due to insufficient data.")
    else:
        print(f"Warning: Binary covariate '{var}' not found in DataFrame.")

if balance_results_binary_pre:
    balance_df_binary_pre = pd.DataFrame(balance_results_binary_pre)
    print(balance_df_binary_pre)
else:
    print("No binary covariates to check or insufficient data.")

Pre-match - Treated group size: 286
Pre-match - Control group size: 347

--- Continuous Covariate Balance Before Matching ---
            Covariate Mean Treated Mean Control Std Diff (SMD) T-Statistic  \
0       current_price       358.51       377.95         -0.081       -1.06   
1  total_review_count        39.42        13.00          0.168        1.92   

  P-Value  
0   0.288  
1   0.056  

--- Binary Covariate Balance Before Matching ---
                        Covariate Prop Treated Prop Control Std Diff (SMD)  \
0               has_free_delivery         0.61         0.75         -0.305   
1              is_verified_seller         0.20         0.19          0.023   
2  has_additional_discount_coupon         0.00         0.00            nan   

  Z-Statistic P-Value  
0       -3.80   0.000  
1        0.29   0.773  
2         nan     nan  


/var/folders/5p/6h8y8yzn7rxdgn4rkplx53rh0000gn/T/ipykernel_3270/2940793195.py:71: RuntimeWarning: invalid value encountered in scalar divide
  smd_binary = (prop_treated - prop_control) / np.sqrt((prop_treated * (1 - prop_treated) + prop_control * (1 - prop_control)) / 2)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std


## Step 2: Run Naive Regression (Without Matching)

We'll run a simple OLS regression to estimate the association between having a high discount and product star rating *before* matching, including all covariates. This provides a baseline.

In [4]:
# Construct the formula for regression
# Ensure all covariates_list items are in df.columns
valid_covariates_for_formula = [cov for cov in covariates_list if cov in df.columns]
formula_unmatched = f"{outcome_variable} ~ {treatment_variable} + {' + '.join(valid_covariates_for_formula)}"
print(f"Unmatched regression formula: {formula_unmatched}")

try:
    model_unmatched = sm.OLS.from_formula(formula_unmatched, data=df).fit()
    print("\n--- Naive Regression (Unmatched Data) ---")
    print(model_unmatched.summary())
except Exception as e:
    print(f"Error running unmatched regression: {e}")
    print("This might be due to perfect multicollinearity or insufficient data/variation.")

Unmatched regression formula: rating ~ is_high_discount + current_price + has_free_delivery + is_verified_seller + total_review_count + has_additional_discount_coupon

--- Naive Regression (Unmatched Data) ---
                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     21.60
Date:                Mon, 26 May 2025   Prob (F-statistic):           5.63e-20
Time:                        16:48:14   Log-Likelihood:                -1384.6
No. Observations:                 633   AIC:                             2781.
Df Residuals:                     627   BIC:                             2808.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1965: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


## Step 3: Propensity Score Matching

Now, we will perform Propensity Score Matching to create more comparable groups.

1.  **Estimate Propensity Scores**: Use logistic regression to estimate the probability of a product having a 'High Discount' based on its covariates.
2.  **Perform Matching**: Match each 'High Discount' product to one 'Low/No Discount' product with the closest propensity score (Nearest Neighbor Matching), potentially using a caliper to ensure match quality.

In [5]:
# Define covariates for propensity score model
# Ensure all covariates_list items are in df.columns
X_psm = df[valid_covariates_for_formula].copy() # Features for PSM
y_psm = df[treatment_variable].copy()        # Treatment indicator

# Handle potential NaNs in features for PSM (LogisticRegression doesn't like NaNs)
# This should have been handled in data prep, but as a safeguard:
X_psm.fillna(X_psm.median(), inplace=True) # Example: median imputation for numeric
# For binary, mode imputation or careful handling is needed if not already done.
# Ensure all covariates are numeric for sklearn's LogisticRegression (dummy encode if needed)

# 1. Estimate propensity scores using logistic regression
logit_model = LogisticRegression(solver='liblinear', max_iter=1000, class_weight='balanced') # Added class_weight for imbalanced treatment
try:
    logit_model.fit(X_psm, y_psm)
    df['propensity_score'] = logit_model.predict_proba(X_psm)[:, 1]
except ValueError as e:
    print(f"Error fitting Logistic Regression for propensity scores: {e}")
    print("Check for NaNs or non-numeric data in X_psm, or if a class in y_psm is missing.")
    # If error, create dummy propensity_score to allow rest of code to run for structure check
    df['propensity_score'] = np.random.rand(len(df))


# 2. Perform Matching
treated_units = df[df[treatment_variable] == 1].copy()
control_units = df[df[treatment_variable] == 0].copy()

if treated_units.empty or control_units.empty or 'propensity_score' not in control_units.columns:
    print("Cannot perform matching: Not enough data in treatment/control groups or propensity scores missing.")
    matched_df = pd.DataFrame() # Empty df if matching fails
else:
    # Ensure control_units for NearestNeighbors fit doesn't have NaNs in pscore
    control_units_nn = control_units[['propensity_score']].copy()
    control_units_nn.dropna(inplace=True) # Should be handled earlier, but safety
    
    if control_units_nn.empty:
        print("Cannot perform matching: Control units have no valid propensity scores for KNN.")
        matched_df = pd.DataFrame()
    else:
        # Match treated units to control units
        nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
        nn.fit(control_units_nn[['propensity_score']])
        
        # Ensure treated_units for kneighbors doesn't have NaNs in pscore
        treated_units_nn = treated_units[['propensity_score']].copy()
        treated_units_nn.dropna(inplace=True) # Should be handled earlier
        
        if treated_units_nn.empty:
            print("Cannot perform matching: Treated units have no valid propensity scores for KNN.")
            matched_df = pd.DataFrame()
        else:
            distances, indices = nn.kneighbors(treated_units_nn[['propensity_score']])

            # Apply caliper to ensure match quality (optional but recommended)
            CALIPER = 0.05 # Max allowed difference in propensity scores
            matched_indices_in_control = indices.flatten()
            
            # Filter out matches outside the caliper
            # Ensure indices are valid before trying to use them
            valid_matches = []
            original_treated_indices = treated_units_nn.index # Get original indices of treated units used for NN

            for i in range(len(original_treated_indices)):
                treated_idx = original_treated_indices[i]
                control_original_idx = control_units_nn.index[matched_indices_in_control[i]] # Get original index of matched control
                
                if distances[i][0] <= CALIPER:
                    valid_matches.append((treated_idx, control_original_idx))
            
            matched_treated_indices = [match[0] for match in valid_matches]
            matched_control_indices = [match[1] for match in valid_matches]

            df_matched_treated = df.loc[matched_treated_indices]
            df_matched_control = df.loc[matched_control_indices]
            
            # Create the matched DataFrame
            matched_df = pd.concat([df_matched_treated, df_matched_control])
            print(f"\nTotal units in matched sample: {matched_df.shape[0]}")
            print(f"Number of treated units in matched sample: {df_matched_treated.shape[0]}")
            print(f"Number of control units in matched sample: {df_matched_control.shape[0]}")

if matched_df.empty:
    print("Warning: Matched DataFrame is empty. Cannot proceed with post-match analysis.")


Total units in matched sample: 562
Number of treated units in matched sample: 281
Number of control units in matched sample: 281


## Step 4: Check Covariate Balance *After* Matching

After matching, we re-evaluate the balance of covariates between the treated and the now-matched control groups. Ideally, any significant differences observed before matching should be substantially reduced.

In [6]:
if not matched_df.empty:
    # Separate matched treated and control groups
    matched_treated_group = matched_df[matched_df[treatment_variable] == 1]
    matched_control_group = matched_df[matched_df[treatment_variable] == 0]

    if matched_treated_group.empty or matched_control_group.empty:
        print("Not enough data in matched treatment or control group to check balance.")
    else:
        print(f"Post-match - Matched Treated group size: {len(matched_treated_group)}")
        print(f"Post-match - Matched Control group size: {len(matched_control_group)}")

        # --- Balance for Continuous Covariates After Matching ---
        print("\n--- Continuous Covariate Balance AFTER Matching ---")
        balance_results_continuous_post = []
        for var in continuous_covariates:
            if var in matched_df.columns:
                if not matched_treated_group[var].dropna().empty and not matched_control_group[var].dropna().empty:
                    mean_treated_post = matched_treated_group[var].mean()
                    mean_control_post = matched_control_group[var].mean()
                    std_treated_post = matched_treated_group[var].std()
                    std_control_post = matched_control_group[var].std()
                    
                    s_pooled_post = np.sqrt(((len(matched_treated_group)-1)*(std_treated_post**2) + \
                                            (len(matched_control_group)-1)*(std_control_post**2)) / \
                                        (len(matched_treated_group) + len(matched_control_group) - 2))
                    smd_post = (mean_treated_post - mean_control_post) / s_pooled_post if s_pooled_post > 0 else 0

                    t_stat_post, p_val_post = ttest_ind(matched_treated_group[var].dropna(), matched_control_group[var].dropna(), equal_var=False)
                    balance_results_continuous_post.append({
                        'Covariate': var,
                        'Mean Treated': f"{mean_treated_post:.2f}",
                        'Mean Control': f"{mean_control_post:.2f}",
                        'Std Diff (SMD)': f"{smd_post:.3f}",
                        'T-Statistic': f"{t_stat_post:.2f}",
                        'P-Value': f"{p_val_post:.3f}"
                    })
                else:
                    print(f"Skipping post-match t-test for {var} due to insufficient data.")
            else:
                print(f"Warning: Post-match continuous covariate '{var}' not found.")
        
        if balance_results_continuous_post:
            balance_df_continuous_post = pd.DataFrame(balance_results_continuous_post)
            print(balance_df_continuous_post)
        else:
            print("No continuous covariates to check post-match or insufficient data.")


        # --- Balance for Binary Covariates After Matching ---
        print("\n--- Binary Covariate Balance AFTER Matching ---")
        balance_results_binary_post = []
        for var in binary_covariates:
            if var in matched_df.columns:
                if not matched_treated_group[var].dropna().empty and not matched_control_group[var].dropna().empty:
                    prop_treated_post = matched_treated_group[var].mean()
                    prop_control_post = matched_control_group[var].mean()
                    
                    count_treated_post = matched_treated_group[var].sum()
                    n_treated_post = matched_treated_group[var].count()
                    count_control_post = matched_control_group[var].sum()
                    n_control_post = matched_control_group[var].count()

                    if n_treated_post > 0 and n_control_post > 0:
                        smd_binary_post = (prop_treated_post - prop_control_post) / np.sqrt((prop_treated_post * (1 - prop_treated_post) + prop_control_post * (1 - prop_control_post)) / 2)
                        
                        z_stat_post, p_val_post = proportions_ztest([count_treated_post, count_control_post], [n_treated_post, n_control_post])
                        balance_results_binary_post.append({
                            'Covariate': var,
                            'Prop Treated': f"{prop_treated_post:.2f}",
                            'Prop Control': f"{prop_control_post:.2f}",
                            'Std Diff (SMD)': f"{smd_binary_post:.3f}",
                            'Z-Statistic': f"{z_stat_post:.2f}",
                            'P-Value': f"{p_val_post:.3f}"
                        })
                    else:
                        print(f"Skipping post-match z-test for {var} due to zero observations.")
                else:
                    print(f"Skipping post-match balance for {var} due to insufficient data.")
            else:
                print(f"Warning: Post-match binary covariate '{var}' not found.")

        if balance_results_binary_post:
            balance_df_binary_post = pd.DataFrame(balance_results_binary_post)
            print(balance_df_binary_post)
        else:
            print("No binary covariates to check post-match or insufficient data.")
else:
    print("Skipping post-match balance checks as matched_df is empty.")

Post-match - Matched Treated group size: 281
Post-match - Matched Control group size: 281

--- Continuous Covariate Balance AFTER Matching ---
            Covariate Mean Treated Mean Control Std Diff (SMD) T-Statistic  \
0       current_price       359.68       377.35         -0.088       -1.04   
1  total_review_count        10.30         7.41          0.117        1.39   

  P-Value  
0   0.299  
1   0.166  

--- Binary Covariate Balance AFTER Matching ---
                        Covariate Prop Treated Prop Control Std Diff (SMD)  \
0               has_free_delivery         0.60         0.60          0.000   
1              is_verified_seller         0.20         0.19          0.027   
2  has_additional_discount_coupon         0.00         0.00            nan   

  Z-Statistic P-Value  
0        0.00   1.000  
1        0.32   0.749  
2         nan     nan  


/var/folders/5p/6h8y8yzn7rxdgn4rkplx53rh0000gn/T/ipykernel_3270/722815390.py:64: RuntimeWarning: invalid value encountered in scalar divide
  smd_binary_post = (prop_treated_post - prop_control_post) / np.sqrt((prop_treated_post * (1 - prop_treated_post) + prop_control_post * (1 - prop_control_post)) / 2)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std


## Step 5: Estimate Treatment Effect on Matched Sample

With the matched sample, we can now estimate the effect of 'High Discount' on 'Product Star Rating'. We can do this by comparing the means of the outcome variable between the treated and matched control groups, or by running a regression on the matched sample.

In [7]:
if not matched_df.empty and not matched_treated_group.empty and not matched_control_group.empty:
    # Method 1: Direct comparison of means for the outcome variable
    mean_rating_treated = matched_treated_group[outcome_variable].mean()
    mean_rating_control = matched_control_group[outcome_variable].mean()
    print(f"\nMean {outcome_variable} for Matched Treated (High Discount) group: {mean_rating_treated:.3f}")
    print(f"Mean {outcome_variable} for Matched Control (Low/No Discount) group: {mean_rating_control:.3f}")
    
    # Perform t-test for difference in means on the outcome
    t_stat_outcome, p_val_outcome = ttest_ind(
        matched_treated_group[outcome_variable].dropna(),
        matched_control_group[outcome_variable].dropna(),
        equal_var=False # Assuming unequal variances is safer
    )
    print(f"T-test for difference in {outcome_variable} between matched groups:")
    print(f"T-Statistic: {t_stat_outcome:.2f}, P-Value: {p_val_outcome:.3f}")
    print(f"Estimated Average Treatment Effect (ATT) on {outcome_variable}: {mean_rating_treated - mean_rating_control:.3f}")

    # Method 2: Regression on the matched sample
    # This can help control for any residual imbalances in covariates after matching
    formula_matched = f"{outcome_variable} ~ {treatment_variable} + {' + '.join(valid_covariates_for_formula)}"
    print(f"\nMatched regression formula: {formula_matched}")
    
    try:
        model_matched = sm.OLS.from_formula(formula_matched, data=matched_df).fit()
        print("\n--- Regression on Matched Data ---")
        print(model_matched.summary())
        
        # Compare treatment effects
        print("\n--- Comparison of Treatment Effects ---")
        if 'model_unmatched' in locals():
            print(f"Unmatched model '{treatment_variable}' Coef: {model_unmatched.params.get(treatment_variable, 'N/A'):.4f}, P-value: {model_unmatched.pvalues.get(treatment_variable, 'N/A'):.3f}")
        else:
            print("Unmatched model was not successfully run or 'treat' variable not found.")

        print(f"Matched model '{treatment_variable}' Coef: {model_matched.params.get(treatment_variable, 'N/A'):.4f}, P-value: {model_matched.pvalues.get(treatment_variable, 'N/A'):.3f}")

    except Exception as e:
        print(f"Error running matched regression: {e}")
        print("This might be due to perfect multicollinearity or insufficient data/variation in the matched sample.")
else:
    print("Cannot estimate treatment effect as matched_df or matched groups are empty.")


Mean rating for Matched Treated (High Discount) group: 2.284
Mean rating for Matched Control (Low/No Discount) group: 2.353
T-test for difference in rating between matched groups:
T-Statistic: -0.35, P-Value: 0.726
Estimated Average Treatment Effect (ATT) on rating: -0.070

Matched regression formula: rating ~ is_high_discount + current_price + has_free_delivery + is_verified_seller + total_review_count + has_additional_discount_coupon

--- Regression on Matched Data ---
                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     26.08
Date:                Mon, 26 May 2025   Prob (F-statistic):           1.08e-23
Time:                        16:48:31   Log-Likelihood:                -1218.7
No. Observations:                 562   AIC:     

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1965: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


## Step 6: Interpretation and Conclusion

**Summarize Findings:**
* Report the estimated effect of `is_high_discount` on `product_star_rating` from the matched analysis.
* State whether this effect is statistically significant.
* Compare this to the naive (unmatched) estimate if calculated.

**Discuss Limitations:**
* **Data Availability for Covariates**: Explicitly state which of the desired covariates (`has_free_delivery`, `is_verified_seller`, `has_additional_discount_coupon`) were actually available and used. If any were missing, this is a key limitation as they could be important confounders.
* **Discount Data Reliability**: Discuss any challenges in defining or calculating `discount_percentage` and `is_high_discount` (e.g., issues with `old_price`).
* **Unobserved Confounders**: PSM only controls for *observed* covariates. There might be other unobserved factors that influence both a product's discount strategy and its star rating (e.g., product quality not captured by covariates, seller's marketing effort, specific inventory levels).
* **Cross-Sectional Data**: This analysis is based on a snapshot. We cannot infer changes over time or the precise timing of when discounts were applied relative to when ratings were given (beyond what the overall rating represents).
* **Category Specificity**: Results are specific to the chosen product category and may not generalize.
* **Small Sample Size**: If your final matched sample is small, the estimates might be imprecise or unstable.

**Conclusion**:
Based on the analysis, provide a tentative conclusion about the relationship between offering a high discount and product star ratings for the specific category analyzed, keeping all limitations in mind. For example: "Within the [Your Category] category, after attempting to control for [list your covariates], products with a high discount percentage were found to have, on average, [X points higher/lower/no significant difference in] star ratings compared to matched products with lower/no discounts. However, this finding should be interpreted with caution due to [mention key limitations]."

## Part 3: Prescriptive Analytics with a Large Language Model (LLM) 🤖

This section outlines how we will use the OpenAI API to interact with a Large Language Model (LLM) like GPT-3.5-turbo or GPT-4. The goal is to generate actionable, prescriptive recommendations based on the diagnostic findings from our causal inference analysis (Part 2 of the assignment). We aim to address a specific challenge identified from our results, fulfilling the "Basic Expectation" and touching upon the "Additional Criteria" for Part 3 of the assignment.

---

### 1. Configuration and API Key Setup 🔑

First, we need to import the necessary `openai` library and configure our API key.

**Security Note**: Your OpenAI API key is sensitive. It's crucial to manage it securely. In a production or shared environment, you should use environment variables or a secure configuration file instead of hardcoding the key directly in the notebook. For this assignment, ensure the key is not publicly shared (e.g., if you push your code to a public repository).

```python
import openai
import pandas as pd # pandas might be useful for handling data summaries if needed

# --- OpenAI API Key Configuration ---
# Replace "YOUR_OPENAI_API_KEY" with your actual key before running.
# It's strongly recommended to load it from an environment variable or a local config file
# for better security, e.g., using:
# import os
# openai.api_key = os.getenv("OPENAI_API_KEY")
# If you haven't set it as an environment variable, you can temporarily set it here:

try:
    # PASTE YOUR ACTUAL OPENAI API KEY HERE
    openai_api_key = "YOUR_OPENAI_API_KEY" 
    if not openai_api_key or openai_api_key == "YOUR_OPENAI_API_KEY":
        raise ValueError("API key not set or still a placeholder. Please provide your actual OpenAI API key.")
    openai.api_key = openai_api_key
    print("OpenAI API key configured.")
except ValueError as e:
    print(f"Error: {e}")
    print("LLM interaction will not work without a valid API key.")
    # Provide a dummy key to allow the notebook to run for structure review, but API calls will fail.
    openai.api_key = "sk-dummykeyforstructuretestingxxxxxxxx"
except Exception as e:
    print(f"An unexpected error occurred during API key configuration: {e}")
    openai.api_key = "sk-dummykeyforstructuretestingxxxxxxxx"

In [ ]:
import openai
import pandas as pd

# --- Configuration ---
# IMPORTANT: Securely manage your API key. Do not commit it to version control.
# Consider using environment variables or a config file.
# For this example, we'll prompt, but in a real script, use a secure method.
try:
    openai.api_key = "YOUR_OPENAI_API_KEY" # Replace with your actual key
    if openai.api_key == "YOUR_OPENAI_API_KEY":
        raise ValueError("Please replace 'YOUR_OPENAI_API_KEY' with your actual OpenAI API key.")
except ValueError as e:
    print(e)
    # Fallback for a placeholder if you don't want to put a real key here during initial drafting
    # The API call will fail if this placeholder is used.
    openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


# --- 1. Summarize Your Diagnostic Findings (from Part 2) ---
# Example based on Scenario 1 (High Discount Negatively Affects Rating)
# REPLACE with your actual findings and category
chosen_category = "Smartphones" # << REPLACE
diagnostic_summary = f"""
Our causal inference analysis (using Propensity Score Matching) on MercadoLibre products
in the '{chosen_category}' category investigated the effect of 'high discount percentages'
(defined as >25%) on 'product star ratings' (1-5 stars).
Covariates controlled for included: current price, free delivery status,
verified seller status, total review count, and presence of additional discount coupons.

Key Finding: We found a statistically significant negative effect.
Products with high discounts had, on average, a {0.35:.2f} point lower star rating
(e.g., average rating of 4.15 for high discount group vs. 4.50 for matched low/no discount group, p < 0.05)
compared to similar products with low or no discounts.
This suggests that while discounts might drive sales, they could be harming perceived product quality
or customer satisfaction post-purchase in this category.
"""
# << REPLACE THE FINDING DETAILS (0.35 points, 4.15 vs 4.50, p-value) WITH YOUR ACTUAL RESULTS

# --- 2. Define Your Prescriptive Problem ---
# Example based on Scenario 1
prescriptive_question = f"""
Given our finding that high discounts negatively impact star ratings for '{chosen_category}'
on MercadoLibre, what specific, actionable strategies can sellers of '{chosen_category}'
implement to mitigate this negative impact on ratings when they utilize high discount strategies?
We are looking for recommendations related to product presentation, communication with buyers,
post-purchase engagement, and ways to frame discounts that might preserve or enhance perceived value.
Consider that data insights are based on listings' current price, discount visibility, seller status,
free delivery, additional coupons, and overall rating/review count.
"""

# --- 3. Prepare Context from Your Collected Data (Brief Summary) ---
# This part is about giving the LLM a bit more context.
# You won't "train" it on the raw CSV in a few API calls.
# Instead, provide aggregated insights or typical product profiles.

# Example:
data_context_summary = f"""
Products in the '{chosen_category}' category on MercadoLibre typically range
from $150 to $1200. Discounts, when applied, can be substantial (10-50%).
Many listings offer free delivery. Both verified and unverified sellers operate.
Review counts vary widely. Additional coupons are sometimes present.
Customers value [e.g., battery life, camera quality, screen resolution - IF YOU KNOW THIS FROM YOUR DATA/EXPLORATION].
The platform allows sellers to have detailed product descriptions and images.
"""
# << CUSTOMIZE THIS CONTEXT BASED ON YOUR EXPLORATORY DATA ANALYSIS

# --- 4. Craft the Prompt for the LLM ---
prompt = f"""
**Background & Diagnostic Findings:**
{diagnostic_summary}

**Data Context for '{chosen_category}' on MercadoLibre:**
{data_context_summary}

**Prescriptive Analytics Question:**
{prescriptive_question}

Please provide a set of 3-5 concrete, actionable recommendations for sellers.
For each recommendation, briefly explain the rationale and how it addresses the problem.
Focus on strategies that sellers can realistically implement.
"""

print("--- Sending Prompt to LLM ---")
print(prompt[:500] + "...\n[Prompt Truncated for Display]") # Print a part of the prompt

# --- 5. Use Python to Connect to an LLM (OpenAI API Call) ---
# Ensure you have the 'openai' library installed: pip install openai
# This is a conceptual example. Refer to the latest OpenAI API documentation for exact usage.
try:
    # Note: The 'chat.completions.create' is the modern way for GPT-3.5-turbo and GPT-4
    # Ensure your openai library is up-to-date (pip install --upgrade openai)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo", # Or "gpt-4" if you have access and prefer it
        messages=[
            {"role": "system", "content": "You are an expert e-commerce strategy consultant providing prescriptive analytics advice."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,  # Adjust as needed
        n=1,
        stop=None,
        temperature=0.7, # Adjust for more creative (higher) or factual (lower) responses
    )
    
    llm_recommendations = response.choices[0].message.content.strip()
    
    print("\n--- LLM Generated Prescriptive Recommendations ---")
    print(llm_recommendations)

except Exception as e:
    print(f"\nAn error occurred with the OpenAI API call: {e}")
    print("Please ensure your API key is correct and you have credits.")
    llm_recommendations = "Error generating recommendations. Please check API key and connection."

# --- 6. (For your report) Summary of LLM's Recommendations and Team's Reflection ---
# This part you'll write based on the output.
# e.g., "The LLM suggested focusing on X, Y, Z. We found these recommendations insightful because...
# However, recommendation Y might be challenging to implement due to..."